In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pathlib import Path
import sys
sys.path.insert(0, "..")

import cv2
import dlib
import numpy as np
from tqdm import tqdm

from data import plot, Process300VW
from demo import extract
from utils import constants, data_utils, personal_constants

In [ ]:
base_path = Path('/home/klaus/dev/5_master/5204PRAI6Y-project-ai/data/local_data')
input_path = base_path / 'person_raw'
temp_path = base_path / 'person_temp'
output_path = base_path / f'person_processed_dim{constants.DATASET_300VW_IMSIZE}'
predictor_path = './local_data/shape_predictor_68_face_landmarks.dat'
FPS = 30

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

In [ ]:
all_videos = sorted([p for p in input_path.iterdir() if p.is_dir()])

In [ ]:
for video_dir in tqdm(all_videos):
    temp_dir = temp_path / video_dir.stem

    if not temp_dir.exists():
        temp_path.mkdir(parents=True, exist_ok=True)
        video_path = video_dir / 'video.webm'
        video = cv2.VideoCapture(str(video_path))
        counter = 1
        success, image = video.read()
        while success == 1:
            frame_output_path = temp_dir / 'images' / f'{counter:06d}.jpg'
            frame_output_path.parent.mkdir(parents=True, exist_ok=True)
            cv2.imwrite(str(frame_output_path), image, [int(cv2.IMWRITE_JPEG_QUALITY), constants.DATASET_300VW_IMAGE_QUALITY])
            counter += 1
            success, image = video.read()

In [ ]:
all_frames = sorted([p for p in temp_path.iterdir() if p.is_dir()])
n_images_per_video = [
    len(list((frames_path / 'images').glob('*.jpg')))
    for frames_path in tqdm(all_frames, desc='frame')
]
n_images = sum(n_images_per_video)

print(f'n images: {n_images}')

In [ ]:
switch_train_test_after_n = FPS * 10

In [ ]:
for frames_input_path in tqdm(all_frames, desc='video'):
    train_output_dir = output_path / (frames_input_path.stem + '_train')
    train_frame_output_dir = train_output_dir / 'images'
    train_landmarks_output_path = train_output_dir / 'annotations.npy'
    train_frame_output_dir.mkdir(parents=True, exist_ok=True)
    train_landmarks_output_path.parent.mkdir(parents=True, exist_ok=True)
    train_landmarks = np.empty((n_images, constants.DATASET_300VW_N_LANDMARKS, 2))
    train_counter = 1
    
    test_output_dir = output_path / (frames_input_path.stem + '_test')
    test_frame_output_dir = test_output_dir / 'images'
    test_landmarks_output_path = test_output_dir / 'annotations.npy'
    test_frame_output_dir.mkdir(parents=True, exist_ok=True)
    test_landmarks_output_path.parent.mkdir(parents=True, exist_ok=True)    
    test_landmarks = np.empty((n_images, constants.DATASET_300VW_N_LANDMARKS, 2))
    test_counter = 1
    
    is_train_image = True
    for image_input_path in tqdm(sorted(list((frames_input_path / 'images').glob('*.jpg'))),
                                 desc='frame', leave=False):
        
        image = cv2.imread(str(image_input_path))
        bounding_boxes = detector(image, 1)
        dlib_landmarks = [
            predictor(image, rectangle).parts() for rectangle in bounding_boxes
        ]
        if len(dlib_landmarks) != 1 or len(dlib_landmarks[0]) != constants.DATASET_300VW_N_LANDMARKS:
            continue
        image_landmarks = np.asarray([(lm.x, lm.y) for lm in dlib_landmarks[0]], dtype=float)

        image_box = data_utils.landmarks_to_box(image_landmarks, image.shape)
        extracted_image = data_utils.extract(image, image_box)
        extracted_landmarks = data_utils.offset_landmarks(image_landmarks, image_box)

        output = Process300VW._rescale_image(extracted_image)
        output_landmarks = data_utils.rescale_landmarks(
            extracted_landmarks, extracted_image.shape, constants.DATASET_300VW_IMSIZE
        )

        # if train_counter == 1 and test_counter == 1:
#             plot(image, image_landmarks, image_box)
#             plot(output, output_landmarks)
        
        if is_train_image:
            frame_output_path = train_frame_output_dir
            counter = train_counter
        else:
            frame_output_path = test_frame_output_dir
            counter = test_counter
        
        frame_output_path = frame_output_path / f'{counter:06d}.jpg'
        cv2.imwrite(str(frame_output_path), output, [int(cv2.IMWRITE_JPEG_QUALITY), constants.DATASET_300VW_IMAGE_QUALITY])
        
        # index at counter -1 because they start at 1
        if is_train_image:
            train_landmarks[train_counter-1] = output_landmarks
            train_counter += 1
        else:
            test_landmarks[test_counter-1] = output_landmarks
            test_counter += 1
        
        # -2 because they start at 1
        if ((train_counter + test_counter - 2) % switch_train_test_after_n) == 0:
            is_train_image = not is_train_image
        

    train_landmarks = train_landmarks[:train_counter]
    test_landmarks = test_landmarks[:test_counter]
    np.save(str(train_landmarks_output_path), train_landmarks)
    np.save(str(test_landmarks_output_path), test_landmarks)